In [2]:
import requests
import pandas as pd
import xlwings as xw
import time as tm

url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"

headers = {"Accept-Encoding":"gzip, deflate, br",
           "Accept-Language":"en-US,en;q=0.9",
           "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
}

session = requests.Session()
data  = session.get(url,headers=headers).json()["records"]["data"]

ocdata = []

for i in data:
    for j,k in i.items():
        if j == 'CE' or j =='PE':
            info = k
            info["instrument_type"] = j
            ocdata.append(info)       
df = pd.DataFrame(ocdata)

In [3]:
oc= pd.DataFrame(ocdata)

In [4]:
oc

,strikePrice,expiryDate,underlying,identifier,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrument_type
0,11000,28-Dec-2023,NIFTY,OPTIDXNIFTY28-12-2023PE11000.00,9.5,0,0.0,0,0.0,2.25,0.0,0.0,1550,0,200,3.05,0,0.00,19435.3,PE
1,11000,28-Dec-2023,NIFTY,OPTIDXNIFTY28-12-2023CE11000.00,27.0,0,0.0,0,0.0,8500.00,0.0,0.0,150,50,50,8350.05,50,8700.00,19435.3,CE
2,12000,28-Sep-2023,NIFTY,OPTIDXNIFTY28-09-2023PE12000.00,2.0,0,0.0,0,0.0,2.35,0.0,0.0,15700,2400,1000,1.10,1000,2.95,19435.3,PE
3,12000,28-Sep-2023,NIFTY,OPTIDXNIFTY28-09-2023CE12000.00,1.0,0,0.0,0,0.0,7850.00,0.0,0.0,2000,4200,1750,7000.50,1450,8016.85,19435.3,CE
4,12000,31-Dec-2026,NIFTY,OPTIDXNIFTY31-12-2026PE12000.00,12.0,0,0.0,0,0.0,250.10,0.0,0.0,200,0,200,0.10,0,0.00,19435.3,PE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064,25000,26-Jun-2025,NIFTY,OPTIDXNIFTY26-06-2025CE25000.00,0.0,0,0.0,0,0.0,0.00,0.0,0.0,50,0,50,0.20,0,0.00,19435.3,CE
1065,25000,24-Dec-2025,NIFTY,OPTIDXNIFTY24-12-2025CE25000.00,0.0,0,0.0,0,0.0,0.00,0.0,0.0,2350,0,500,0.60,0,0.00,19435.3,CE
1066,25000,25-Jun-2026,NIFTY,OPTIDXNIFTY25-06-2026CE25000.00,0.0,0,0.0,0,0.0,0.00,0.0,0.0,50,0,50,0.20,0,0.00,19435.3,CE
1067,25000,30-Dec-2027,NIFTY,OPTIDXNIFTY30-12-2027PE25000.00,0.0,0,0.0,0,0.0,0.00,0.0,0.0,50,0,50,0.10,0,0.00,19435.3,PE


In [5]:
columns_name = df.columns.tolist()

In [6]:
columns_name

['strikePrice',
 'expiryDate',
 'underlying',
 'identifier',
 'openInterest',
 'changeinOpenInterest',
 'pchangeinOpenInterest',
 'totalTradedVolume',
 'impliedVolatility',
 'lastPrice',
 'change',
 'pChange',
 'totalBuyQuantity',
 'totalSellQuantity',
 'bidQty',
 'bidprice',
 'askQty',
 'askPrice',
 'underlyingValue',
 'instrument_type']

In [7]:
selected_columns = ['strikePrice',
 'expiryDate','openInterest',
 'changeinOpenInterest',
 'instrument_type',
 'lastPrice',
 'totalTradedVolume'
 ]

In [8]:
df1 = df[selected_columns]

In [9]:
grouped = df1.groupby('instrument_type')
grouped

In [10]:
ce_group = grouped.get_group('CE')
pe_group = grouped.get_group('PE')
ce_filtered = ce_group[ce_group['expiryDate'] == '31-Aug-2023']
pe_filtered = pe_group[pe_group['expiryDate'] == '31-Aug-2023']

In [11]:
ce_ordered = ['openInterest','changeinOpenInterest','lastPrice','strikePrice','totalTradedVolume']
ce = df[ce_ordered]


In [12]:
stradle = ce['totalTradedVolume'].max()
stradle

3701931

In [13]:
pe_ordered = ['lastPrice','changeinOpenInterest','openInterest',]
pe = df[pe_ordered]
pe

,lastPrice,changeinOpenInterest,openInterest
0,2.25,0,9.5
1,8500.00,0,27.0
2,2.35,0,2.0
3,7850.00,0,1.0
4,250.10,0,12.0
...,...,...,...
1064,0.00,0,0.0
1065,0.00,0,0.0
1066,0.00,0,0.0
1067,0.00,0,0.0


In [14]:
oc = pd.concat([ce,pe],axis = 1,join = 'inner')

In [15]:
oc

,openInterest,changeinOpenInterest,lastPrice,strikePrice,totalTradedVolume,lastPrice,changeinOpenInterest,openInterest
0,9.5,0,2.25,11000,0,2.25,0,9.5
1,27.0,0,8500.00,11000,0,8500.00,0,27.0
2,2.0,0,2.35,12000,0,2.35,0,2.0
3,1.0,0,7850.00,12000,0,7850.00,0,1.0
4,12.0,0,250.10,12000,0,250.10,0,12.0
...,...,...,...,...,...,...,...,...
1064,0.0,0,0.00,25000,0,0.00,0,0.0
1065,0.0,0,0.00,25000,0,0.00,0,0.0
1066,0.0,0,0.00,25000,0,0.00,0,0.0
1067,0.0,0,0.00,25000,0,0.00,0,0.0
